In [ ]:
import requests
import urllib.request
from selenium import webdriver
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import StaleElementReferenceException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time
import os
import re
import sys
import pydub
import speech_recognition as sr
# import pyautogui
import math
import warnings
warnings.filterwarnings('ignore')
from selenium.webdriver.common.action_chains import ActionChains

In [ ]:
driver = webdriver.Chrome(r"C:\Upsilonsers\swast\Downloads\chromedriver_win32\chromedriver.exe")
driver.get('http://www.ceo.kerala.gov.in/electoralrolls.html')
driver.maximize_window()

In [ ]:
def delay(driver,waiting_time=4):
    driver.implicitly_wait(waiting_time)

In [ ]:
l1=[]
l2=[]
for i in range (1,140):
    j=1
    while(j>0):
        l1.append(i)
        l2.append(j)
        driver.get('http://www.ceo.kerala.gov.in/pdf/voterslist/2023/FINAL/ML/AC00%s/S11A%sP%s.pdf'%(i,i,j))
        b_text = driver.find_element("xpath",'/html/body/div[2]/div[3]/form/table/tbody/tr[3]/td').text
        j+=1
        if(b_text==''):
            break
        else:
            frames = driver.find_elements(By.TAG_NAME,'iframe')
            recaptcha_control_frame = None
            recaptcha_challenge_frame = None
            for index, frame in enumerate(frames):
                if re.search('reCAPTCHA', frame.get_attribute("title")):
                    recaptcha_control_frame = frame

                if re.search('recaptcha challenge', frame.get_attribute("title")):
                    recaptcha_challenge_frame = frame
            if not (recaptcha_control_frame and recaptcha_challenge_frame):
                print("[ERR] Unable to find recaptcha. Abort solver.")
#             sys.exit()
            # switch to recaptcha frame
            #                 time.sleep(1)
            frames = driver.find_elements(By.TAG_NAME,'iframe')
            driver.switch_to.frame(recaptcha_control_frame)
            # click on checkbox to activate recaptcha
            driver.find_element(By.CLASS_NAME,"recaptcha-checkbox-border").click()
            #pyautogui.click(530, 950)

#             try:
            time.sleep(1)
            #print('hello1')
            try:
                driver.switch_to.default_content()
                frames = driver.find_elements(By.TAG_NAME,'iframe')
                driver.switch_to.frame(recaptcha_challenge_frame)

                # click on audio challenge
                time.sleep(2)
                driver.find_element(By.ID,"recaptcha-audio-button").click()

                # switch to recaptcha audio challenge frame
                driver.switch_to.default_content()
                #                     time.sleep(1)
                frames = driver.find_elements(By.TAG_NAME,'iframe')
                driver.switch_to.frame(recaptcha_challenge_frame)

                # get the mp3 audio file
                delay(driver)
                src = driver.find_element(By.ID,"audio-source").get_attribute("src")
                #print(f"[INFO] Audio src: {src}")

                path_to_mp3 = os.path.normpath(os.path.join(os.getcwd(), "sample.mp3"))
                path_to_wav = os.path.normpath(os.path.join(os.getcwd(), "sample.wav"))

                # download the mp3 audio file from the source
                urllib.request.urlretrieve(src, path_to_mp3)
                time.sleep(1)

            # load downloaded mp3 audio file as .wav
    #                 try:
                sound = pydub.AudioSegment.from_mp3(path_to_mp3)
                sound.export(path_to_wav, format="wav")
                sample_audio = sr.AudioFile(path_to_wav)
    #                 except Exception:
    #                     sys.exit(
    #                         "[ERR] Please run program as administrator or download ffmpeg manually, "
    #                         "https://blog.gregzaal.com/how-to-install-ffmpeg-on-windows/"
    #                     )

            # translate audio to text with google voice recognition
                delay(driver)
                r = sr.Recognizer()
                with sample_audio as source:
                    audio = r.record(source)
                key = r.recognize_google(audio,language='en-IN')
                #print(f"[INFO] Recaptcha Passcode: {key}")

                # key in results and submit
                delay(driver)
                driver.find_element(By.ID,"audio-response").send_keys(key.lower())
                driver.find_element(By.ID,"audio-response").send_keys(Keys.ENTER)
                time.sleep(2)
                driver.switch_to.default_content()
                time.sleep(2)
                down=driver.find_element("xpath","/html/body/div[2]/div[3]/form/table/tbody/tr[5]/td/input")
                down.click()
                time.sleep(1)
            except:
                down=driver.find_element("xpath","/html/body/div[2]/div[3]/form/table/tbody/tr[5]/td/input")
                down.click()
                time.sleep(1)
#             except:
#                 print('lol')
#                 try:
#                     driver.switch_to.default_content()
#                     down=driver.find_element("xpath","/html/body/div[2]/div[3]/form/table/tbody/tr[5]/td/input")
#                     down.click()
#                     time.sleep(1)
#                 except:
#                     raise e
#                     pass

In [ ]:
key

In [ ]:
l2

In [ ]:
b_text

In [ ]:
sample_audio = sr.AudioFile(r"C:\Users\swast\Downloads\audio (1).wav")
delay(driver)
r = sr.Recognizer()
with sample_audio as source:
    audio = r.record(source)
key = r.recognize_google(audio,language='en-IN')